In [220]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [221]:
dataset = pd.read_excel('../data/boxScores.xlsx')


In [222]:
# Filter out rows where the 'MIN' column is zero
dataset = dataset[dataset['MIN'] != 0]
dataset = dataset.dropna(subset=['W/L', 'Last3_FP_Avg'])
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,STL,BLK,TOV,PF,+/-,FP,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
count,9601,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,...,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000,9601.000000
mean,2024-11-24 10:02:47.232580096,22.773774,10.673367,3.892095,8.373607,1.281221,3.553692,1.607958,2.060306,1.022289,...,0.798563,0.482658,1.332361,1.800958,0.012707,21.882262,21.648995,21.555911,21.483699,21.171419
min,2024-10-24 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-43.000000,-3.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2024-11-08 00:00:00,15.000000,4.000000,1.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,-7.000000,10.000000,11.700000,11.900000,11.885714,11.666667
50%,2024-11-23 00:00:00,24.000000,9.000000,3.000000,7.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,2.000000,0.000000,20.100000,20.266667,20.200000,20.085714,19.850000
75%,2024-12-08 00:00:00,32.000000,16.000000,6.000000,12.000000,2.000000,5.000000,2.000000,3.000000,2.000000,...,1.000000,1.000000,2.000000,3.000000,7.000000,31.400000,30.000000,29.840000,29.614286,29.108333
max,2024-12-29 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,8.000000,10.000000,10.000000,6.000000,43.000000,98.300000,80.500000,75.360000,72.200000,72.200000
std,NaN,10.732722,8.828833,3.240952,6.022950,1.543626,3.182514,2.248387,2.713893,1.351359,...,1.008424,0.846522,1.455966,1.447722,11.366986,14.958324,13.077532,12.690014,12.530303,12.270331


In [223]:
# Filter out players who aren't in clusters for accuracy
clusterdf = pd.read_excel('../data/clusteredPlayers.xlsx')
clusterdf = clusterdf.drop('Unnamed: 0', axis=1)
clusterDict = {}
clusterDict.update(pd.Series(clusterdf['Cluster'].values, index=clusterdf['Player']).to_dict())
dataset['CLUSTER'] = dataset['PLAYER'].map(clusterDict).astype('category')
dataset = dataset.dropna(subset=['CLUSTER'])
dataset.sort_values(by=['GAME DATE'], ascending = [False], inplace=True)
len(dataset)

7385

In [224]:
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,STL,BLK,TOV,PF,+/-,FP,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg
count,7385,7385.000000,7385.00000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,...,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000,7385.000000
mean,2024-11-24 06:53:34.407582976,26.539201,12.75694,4.644144,9.946107,1.533243,4.215843,1.935410,2.459851,1.162898,...,0.929722,0.551794,1.564387,2.042248,0.227624,25.851117,25.614999,25.534631,25.471709,25.138309
min,2024-10-24 00:00:00,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-43.000000,-2.800000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2024-11-08 00:00:00,20.000000,6.00000,2.000000,6.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,-8.000000,15.300000,16.933333,17.100000,17.200000,16.968182
50%,2024-11-23 00:00:00,27.000000,11.00000,4.000000,9.000000,1.000000,4.000000,1.000000,2.000000,1.000000,...,1.000000,0.000000,1.000000,2.000000,0.000000,24.500000,24.166667,24.160000,24.066667,23.638889
75%,2024-12-08 00:00:00,33.000000,18.00000,7.000000,14.000000,2.000000,6.000000,3.000000,4.000000,2.000000,...,1.000000,1.000000,2.000000,3.000000,8.000000,34.700000,32.933333,32.520000,32.366667,31.900000
max,2024-12-29 00:00:00,48.000000,60.00000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,8.000000,10.000000,10.000000,6.000000,43.000000,98.300000,80.500000,75.360000,72.200000,72.200000
std,NaN,8.522967,8.74188,3.213064,5.788720,1.617303,3.226338,2.390115,2.867429,1.424377,...,1.056280,0.899804,1.516684,1.429303,12.304970,14.163941,11.891585,11.401002,11.189650,10.885811


For reference, features are the raw input data provided to the model to generate predictions, while the label is the target outcome that the model aims to predict. These above features can and probably should be tweaked and adjusted to improve the model's performance.

In [225]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfLabels = dataset[labelName]

In [226]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)

** Note numpy array is only numbers while pandas dataframe can be a mix of numbers and strings.

In [227]:
# Scaling the data
x = features
x = StandardScaler().fit_transform(x)

In [228]:
dataset = dataset[['PLAYER', 'GAME DATE', 'FP', 'Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']]
players = dataset['PLAYER']
gameDates = dataset['GAME DATE']

In [229]:
train, test, trainLabels, testLabels, trainPlayers, testPlayers, trainDates, testDates= train_test_split(features, labels, players, gameDates, test_size=0.25, random_state=4)
testPlayers = testPlayers.reset_index(drop=True)
testDates = testDates.reset_index(drop=True)


In [230]:
rf = RandomForestRegressor(random_state=4)

In [231]:
import pickle
rf.fit(train, trainLabels)
with open("../models/RFFullNBA.sav", 'wb') as f:
    pickle.dump(rf, f)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [232]:
train_predictions = rf.predict(train)
predictions = rf.predict(test)

In [233]:
df = pd.DataFrame(test, columns = featureNames)
df['PLAYER'] = testPlayers
df['GAME DATE'] = testDates
df['actual'] = testLabels
df['predicted'] = predictions
df['error'] = abs(df['actual'] - df['predicted'])
df = df[['PLAYER'] + [col for col in df.columns if col != 'PLAYER']]

In [234]:
df.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,GAME DATE,actual,predicted,error
count,1847.000000,1847.000000,1847.000000,1847.000000,1847,1847.000000,1847.000000,1847.000000
mean,25.749919,25.733817,25.686125,25.417497,2024-11-23 23:23:21.407688192,26.108988,25.893448,8.519803
min,-1.000000,-1.000000,-1.000000,-1.000000,2024-10-24 00:00:00,-2.000000,2.575000,0.003833
25%,17.350000,17.690000,17.742857,17.638889,2024-11-06 00:00:00,16.300000,18.229917,3.362000
50%,24.533333,24.400000,24.400000,24.053333,2024-11-22 00:00:00,25.100000,24.487000,7.102000
75%,32.833333,32.483333,32.358571,31.880797,2024-12-10 00:00:00,34.700000,32.207667,12.337867
max,76.433333,72.200000,70.742857,67.690000,2024-12-29 00:00:00,98.300000,66.460000,54.211000
std,11.706792,11.292215,11.030503,10.765749,NaN,13.871666,10.213121,6.628989


In [235]:
train, test, trainLabels, testLabels = train_test_split(x, labels, test_size=0.25, random_state=4)
rf = RandomForestRegressor(random_state=4)
rf.fit(train, trainLabels)
rfPredNoStandard = rf.predict(test)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [236]:
df4 = pd.DataFrame(test, columns = featureNames)
df4['actual'] = testLabels
df4['predicted'] = rfPredNoStandard
df4['error'] = abs(df4['actual'] - df4['predicted'])

In [237]:
df4.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,actual,predicted,error
count,1847.000000,1847.000000,1847.000000,1847.000000,1847.000000,1847.000000,1847.000000
mean,0.011347,0.017472,0.019163,0.025649,26.108988,25.884485,8.517296
std,0.984527,0.990525,0.985844,0.989038,13.871666,10.214888,6.633194
min,-2.238289,-2.327552,-2.365892,-2.401298,-2.000000,2.580000,0.007000
25%,-0.695076,-0.688112,-0.690761,-0.688964,16.300000,18.231500,3.367092
50%,-0.090967,-0.099527,-0.095783,-0.099676,25.100000,24.453000,7.082617
75%,0.607053,0.609523,0.615509,0.619425,34.700000,32.207667,12.325000
max,4.273759,4.093371,4.046079,3.909178,98.300000,66.460000,54.211000


In [238]:
# pd.DataFrame.to_excel(df, '../data/playerPredictions.xlsx')

PermissionError: [Errno 13] Permission denied: '../data/playerPredictions.xlsx'

In [239]:
dfCheck = pd.DataFrame()
# Difference in accuracy between df3 and df4 is negligible so I am sticking with 3 for better readability
dfCheck['df3'] = df['error']
dfCheck['df4'] = df4['error']
dfCheck.describe()

,df3,df4
count,1847.000000,1847.000000
mean,8.519803,8.517296
std,6.628989,6.633194
min,0.003833,0.007000
25%,3.362000,3.367092
50%,7.102000,7.082617
75%,12.337867,12.325000
max,54.211000,54.211000


Note: Scaling and preprocessing data matters more on larger datasets